[9] **Modelo XY**: El modelo XY es como el modelo de Ising, salvo que los espines pueden apuntar en cualquier dirección y no sólo en 2 direcciones. Por lo tanto, para calcular la energía del sistema se requiere calcular el producto punto (equivalentemente el ángulo) entre el campo magnético y el espín y también el ángulo entre dos espines vecinos. 

$$ H(\sigma) = - \sum_{i\neq j} J_{ij}\; \vec{\sigma}_i\cdot\vec{\sigma}_j -\sum_j \mathbf{B}_j\cdot \vec{\sigma}_j$$

Considera que $J$ y $B$ son constantes.

Este es uno de esos modelos que NO debería presentar una transición de fase en el sentido clásico según el teorema de Mermin-Wagner, pues es un sistema continuo (la posición es discreta, pero la orientación es continua). 

* Repite el primer inciso del problema anterior usando este modelo, pero esta vez para $k_B T = 0.1, 0.9$ y $2$. Con eso obtén el número de pasos de Monte-Carlo para que el sistema "termalice". 

* Mide $\langle m \rangle$ y $\langle E \rangle$ (usando el promedio de los último $10^3$ pasos en un solo sistema) y reduciendo la temperatura desde un estado completamente desordenado (comienza en $k_B T = 2$), para variaciones de $\Delta k_B T = 0.01$ lentas (2 veces el número de pasos de Monte-Carlo que encontraste en el iniciso 1). ¿Si observas una transición de fase (o algo similar), a qué temperatura la observas (aproximadamente)? 

* Repite ahora el experimento con variaciones rápidas de la temeratura (1, 1/2, 1/3, 1/4 y 1/8 veces el número de pasos de Monte-Carlo que usaste en el inciso anterior). Haz una gráfica del sistema que obtienes en cada caso cuando llegas a temperatura 0. Para esto dibuja una red de $50\times 50$ donde en cada nodo pongas una flechita que represente al espín (apuntando en la dirección que le corresponde). ¿Se ordenan siempre los espines en alguna dirección? 

Nota: Este es el segundo problema que trataron Kosterliz y Thoules, el primero fue el de discos duros, el segundo fue este. Por ambos se les dió el premio Nobel de Física en 2016. Además, en 2021 Parisi recibió el premio Nobel de Física por 2 razones, por estudiar sistemas complejos en general y por sus trabajos sobre vidrios de espín, que son las estructuras que deberían encontrar para enfriamientos rápidos en este modelo! 

$$\vec{\sigma_1}\cdot\vec{\sigma_2} = |\vec{\sigma_1}||\vec{\sigma_2}|cos\theta$$
entonces se concluye que cómo $cos \in [-1, 1]$
$$\vec{\sigma_1}\cdot\vec{\sigma_2} \in [-|\vec{\sigma_1}||\vec{\sigma_2}|,|\vec{\sigma_1}||\vec{\sigma_2}|]$$

cómo $|\vec{\sigma_k}| = 1$

$$\vec{\sigma_1}\cdot\vec{\sigma_2} \in [-1, 1]$$

In [1]:
using Distributions

In [2]:
function initialState(L)
    return rand(Uniform(-1,1), L, L)
end

initialState (generic function with 1 method)

In [3]:
function randomSite(L)
    return rand(1:L), rand(1:L)
end

randomSite (generic function with 1 method)

In [4]:
function stateChange(L, state)
    stat1 = copy(state)
    ala = randomSite(L)
    
    stat1[ala[1], ala[2]] *= -1
    return stat1
end

stateChange (generic function with 1 method)

In [5]:
function neighbors(n, m, initialState)
    A = []
    L = size(initialState)[1]
    
    if n == 1 && m == 1
        append!(A, [initialState[L, m], initialState[n, L], initialState[n + 1, m], initialState[n, m+1]])
    elseif n == L && m == 1
        append!(A, [initialState[1, m], initialState[n, L], initialState[n - 1, m], initialState[n, m+1]])
    elseif n == 1 && m == L
        append!(A, [initialState[L, m], initialState[n + 1, L], initialState[n, m - 1], initialState[n, 1]])
    elseif n == L && m == L
        append!(A, [initialState[1, m], initialState[n - 1, m], initialState[n, 1], initialState[n, m-1]])
    elseif n == 1
        append!(A, [initialState[L, m], initialState[n + 1, m], initialState[n, m-1], initialState[n, m+1]])
    elseif n == L
        append!(A, [initialState[1, m], initialState[n - 1, m], initialState[n, m-1], initialState[n, m+1]])
    elseif m == 1
        append!(A, [initialState[n+1, m], initialState[n - 1, m], initialState[n, L], initialState[n, m+1]])
    elseif m == L
        append!(A, [initialState[n+1, m], initialState[n - 1, m], initialState[n, 1], initialState[n, m-1]])
    else
        append!(A, [initialState[n-1, m], initialState[n+1, m], initialState[n, m-1], initialState[n, m+1]])
    end
    
    return A
end

neighbors (generic function with 1 method)

In [6]:
function hamiltonian(state)
    energia = 0
    summ = 0
    l = size(state)[1]
    
    for j in 1:l, k in 1:l
        summ -= sum(state[j, k] * neighbors(j, k, state))
    end
    
    summ -= 0.01*sum(state)
    
    return summ
end

hamiltonian (generic function with 1 method)

In [7]:
function boltzProb(initState, prblState, KT)
    deltaE = hamiltonian(prblState) - hamiltonian(initState)
    
    if deltaE == 0 && KT == 0
        return 0
    else 
        return min(1, exp(-deltaE/KT))
    end
end

boltzProb (generic function with 1 method)

In [8]:
function metropolisXY(n, KT, L = 40)
     state = initialState(L)
     for k in 1:n
        stChn = stateChange(L, state)
        prob = boltzProb(state, stChn, KT)
        r = rand(1)[1]

        if r < prob
            state = stChn
        end

    end
    
    return state
end

metropolisXY (generic function with 2 methods)

In [ ]:
datos0_1m = []
datos2_2m = []
datos4m = []

datos0_1H = []
datos2_2H = []
datos4H = []

maxHamiltonian = hamiltonian(ones(40, 40))
    
for k in 80000:3000:100000
    state0_1 = metropolisXY(k, 0.1)
    state2_2 = metropolisXY(k, 0.9)
    state4 = metropolisXY(k, 2)
    
    append!(datos0_1m, abs(sum(state0_1)))
    append!(datos2_2m, abs(sum(state2_2)))
    append!(datos4m, abs(sum(state4)))

    append!(datos0_1H, hamiltonian(state0_1)/maxHamiltonian)
    append!(datos2_2H, hamiltonian(state2_2)/maxHamiltonian)
    append!(datos4H, hamiltonian(state4)/maxHamiltonian)
end

In [ ]:
using PyPlot
using Seaborn

Seaborn.set_theme()

PyPlot.plot(60000:3000:80000, datos0_1m ./ 1600)

In [ ]:
PyPlot.plot(60000:3000:80000, datos0_1H)

In [ ]:
PyPlot.plot(60000:3000:80000, datos2_2H)

In [ ]:
PyPlot.plot(60000:3000:80000, datos2_2m ./ 1600)

In [ ]:
PyPlot.plot(60000:3000:80000, datos4m ./ 1600)

In [ ]:
PyPlot.plot(60000:3000:80000, datos4H ./ 1600)

al parecer de 60k a 70k es cuando se termaliza el sistema xy

In [9]:
function metropolisXYdatos(n, KT, L = 40)
     state = initialState(L)
     maxHamiltonian = hamiltonian(ones(40, 40))
     DataM = []
     DataH = []
    
     for k in 1:n
        stChn = stateChange(L, state)
        prob = boltzProb(state, stChn, KT)
        r = rand(1)[1]

        if r < prob
            state = stChn
        end
        
        if k > n - 10000 && k%100 == 0
            append!(DataM, abs(sum(state)))
            append!(DataH, hamiltonian(state)/maxHamiltonian)
        end

    end
    
    return DataM, DataH
end

metropolisXYdatos (generic function with 2 methods)

In [ ]:
DatosM = []
DatosH = []

print("ele")
for β in 0.1:0.25:4
    println(β)
    Data = [metropolisXYdatos(75000, β) for k in 1:2] 
    DATOS = (sum([data[1] for data in Data])/2, sum([data[2] for data in Data])/2)
    
    push!(DatosM, DATOS[1])
    push!(DatosH, DATOS[2])
end

ele0.1


In [ ]:
sum([(1, 2), (3, 4)])